# Transportation problem

A transportation problem has nodes that are associated with supply or demand, with lines ("arcs") between the nodes having associated profits or costs. The goal is to minimize or maximize an objective while satisfying the supply and demand constraints.

As long as supplies and demands are integer values, the transportation problem will give an integer solution. We can put upper and lower bounds on variables and still get an integer solution.

## Problem description

*This problem is from the MSBA optimization course.*

There are three customers (A, B, and C) demanding goods from two plants (1, 2). Each customer requires 30 units. Plant 1 has 40 units, while plant 2 has 50 units. Each customer has negotiated a different per-unit costs with each plant. The goal is to ship the materials at minimum total cost.

## Mathematical formulation

- Plants $i$
- Customers $j$
- $C_{i,j}$: Cost to ship one unit from plant $i$ to customer $j$
- $S_{i}$: Supply from plant $i$
- $D_{j}$: Demand of customer $j$

**Variables**:

- $x_{i,j}$: Units shipped from plant $i$ to customer $j$

**Objective**:

Minimize $\sum\nolimits_{i} \sum\nolimits_{j} x_{i,j} \cdot C_{i,j}$

**Constraints**:

1. $\sum\nolimits_{i} x_{i,j} \geq D_{j} \forall j$ (must at least meet demand)
2. $\sum\nolimits_{j} x_{i,j} \leq S_{i} \forall j$ (cannot exceed supply)
3. $x_{i,j} > 0$ (non-negative units)

## PuLP model

In [1]:
import pulp

In [2]:
# Transportation Problem: data
#
Plants = ['Plant 1', 'Plant 2']
Customers = ['Cust A', 'Cust B', 'Cust C']

Cost = {
    ('Plant 1', 'Cust A') : 15,
    ('Plant 1', 'Cust B') : 35,
    ('Plant 1', 'Cust C') : 25,
    ('Plant 2', 'Cust A') : 10,
    ('Plant 2', 'Cust B') : 30,
    ('Plant 2', 'Cust C') : 40
}

Supply = {
    'Plant 1' : 40,
    'Plant 2' : 50
}

Demand = {
    'Cust A' : 30,
    'Cust B' : 30,
    'Cust C' : 30,
}

In [10]:
## Create model as minimization problem
mdl = pulp.LpProblem("TransportationProblem", pulp.LpMinimize)

In [11]:
## Create the foods amount variable
units = pulp.LpVariable.dicts("Units", ((p, c) for p in Plants for c in Customers), lowBound = 0)

In [12]:
## Minimize the total cost of shipping
mdl += pulp.lpSum([units[p,c] * Cost[p,c] for p in Plants for c in Customers])

In [13]:
## Constraint 1: Must meet demand
for c in Customers:
    mdl += (pulp.lpSum(units[p,c] for p in Plants) >= Demand[c], "Demand_"+c)

In [14]:
## Constraint 2: Must not exceed supply
for p in Plants:
    mdl += (pulp.lpSum(units[p,c] for c in Customers) <= Supply[p], "Supply"+p)

In [15]:
mdl.solve()

1

## Solution report

In [16]:
print("Solve status: {:s}".format(pulp.LpStatus[mdl.status]))

Solve status: Optimal


In [17]:
obj = pulp.value(mdl.objective)
print("Min cost: {:.2f}".format(obj))

Min cost: 2000.00


In [22]:
for (p,c) in Cost.keys():
    u = pulp.value(units[p,c])
    if (u > 0):
        print("{:s} to {:s}: {:.0f}".format(p, c, u))

Plant 1 to Cust A: 10
Plant 1 to Cust C: 30
Plant 2 to Cust A: 20
Plant 2 to Cust B: 30


## Problem variant

Let's say plant 1 only has 30 units available, while all other values remain the same. Now we are unable to meet all of the possible demand. Here, we introduce a "dummy" plant to track the unmet demand. Additionally, each plant will incur a shortage cost because of unmet demand.

We can also introduce shipping restrictions such that each plant is contractually required to deliver a certain number of units to certain customers.

## PuLP model

In [23]:
# Transportation Problem: data
#
# For this variant we introduce a 'dummy' plant to represent unmet demand (10 units)
Plants = ['Plant 1', 'Plant 2', 'Plant 3']
Customers = ['Cust A', 'Cust B', 'Cust C']

Cost = {
    ('Plant 1', 'Cust A') : 15,
    ('Plant 1', 'Cust B') : 35,
    ('Plant 1', 'Cust C') : 25,
    ('Plant 2', 'Cust A') : 10,
    ('Plant 2', 'Cust B') : 30,
    ('Plant 2', 'Cust C') : 40,
    ('Plant 3', 'Cust A') : 20,
    ('Plant 3', 'Cust B') : 10,
    ('Plant 3', 'Cust C') : 30
}

Supply = {
    'Plant 1' : 30,
    'Plant 2' : 50,
    'Plant 3' : 10
}

Demand = {
    'Cust A' : 30,
    'Cust B' : 30,
    'Cust C' : 30,
}

In [26]:
## Create model as minimization problem
mdl = pulp.LpProblem("TransportationProblem", pulp.LpMinimize)

## Create the foods amount variable
units = pulp.LpVariable.dicts("Units", ((p, c) for p in Plants for c in Customers), lowBound = 0)

## Minimize the total cost of shipping
mdl += pulp.lpSum([units[p,c] * Cost[p,c] for p in Plants for c in Customers])

## Constraint 1: Must meet demand
for c in Customers:
    mdl += (pulp.lpSum(units[p,c] for p in Plants) >= Demand[c], "Demand_"+c)

## Constraint 2: Must not exceed supply
for p in Plants:
    mdl += (pulp.lpSum(units[p,c] for c in Customers) <= Supply[p], "Supply"+p)
    
# Further restrictions: ship at least 5 units from 1 to B; ship at most 20 units from 1 to C:
mdl += (units['Plant 1','Cust B'] >= 5)
mdl += (units['Plant 1','Cust C'] <= 20)

mdl.solve()

1

In [27]:
print("Solve status: {:s}".format(pulp.LpStatus[mdl.status]))

obj = pulp.value(mdl.objective)
print("Min cost: {:.2f}".format(obj))

for (p,c) in Cost.keys():
    u = pulp.value(units[p,c])
    if (u > 0):
        print("{:s} to {:s}: {:.0f}".format(p, c, u))

Solve status: Optimal
Min cost: 1950.00
Plant 1 to Cust A: 5
Plant 1 to Cust B: 5
Plant 1 to Cust C: 20
Plant 2 to Cust A: 25
Plant 2 to Cust B: 15
Plant 2 to Cust C: 10
Plant 3 to Cust B: 10
